In [6]:
from tqdm.notebook import tqdm

from mlp_mixer.dataset.iterator import ImageNetIterator
from mlp_mixer.layer.mixer import MM
from torch.utils.data import DataLoader

import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
train_iter = ImageNetIterator(root='../imagenet/',
                 height=256,
                 width=256,
                 is_train=True,
                 in_memory=False,
                 verbose=False)

test_iter = ImageNetIterator(root='../imagenet/',
                 height=256,
                 width=256,
                 is_train=True,
                 in_memory=False,
                 verbose=False)

train_loader = DataLoader(train_iter, batch_size=64*2, shuffle=True, num_workers=10)
valid_loader = DataLoader(test_iter, batch_size=64*2, shuffle=False, num_workers=10)

In [3]:
h, w, c = 256, 256, 3
p = 32
C=  256
S = (h*w)//(p**2)

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
model = nn.DataParallel(MM(h, w, c, 6, p, C, S, 2048, 256, len(train_iter.label_dict)).to(device))

In [7]:
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

In [8]:
def train() : 
    model.train()
    losses = []
    accuracies = []

    for x,y in tqdm(train_loader, desc='train') : 
        pred = model(x.to(device))

        optimizer.zero_grad()
        loss = criterion(pred, y.to(device))
        loss.backward()
        optimizer.step()

        correct = (torch.argmax(pred, dim=1) == y.to(device)).sum()
        acc = correct.item() / y.shape[0]

        accuracies.append(acc)
        losses.append(loss.item())

    agg_acc = sum(accuracies) / len(accuracies)
    agg_loss = sum(losses) / len(losses)
    return agg_acc, agg_loss

def evalulate() : 
    model.eval()
    losses = []
    accuracies = []

    for x,y in tqdm(valid_loader, desc='valid') : 
        pred = model(x.to(device))

        loss = criterion(pred, y.to(device))
        correct = (torch.argmax(pred, dim=1) == y.to(device)).sum()
        acc = correct.item() / y.shape[0]

        accuracies.append(acc)
        losses.append(loss.item())

    agg_acc = sum(accuracies) / len(accuracies)
    agg_loss = sum(losses) / len(losses)
    return agg_acc, agg_loss

In [59]:
epoches = 20

for proc in range(epoches) : 
    t_acc, t_loss = train()
    v_acc, v_loss = evalulate()
    print(f"""
                === {proc+1}th Epoch ===
    
        Train Loss : {round(t_loss, 3)} | Train Acc : {round(t_acc, 3)}
        Valid Loss : {round(v_loss, 3)} | Valid Acc : {round(v_acc, 3)}
        
        ============================================
        ============================================
    """)

train:   0%|          | 0/600 [00:00<?, ?it/s]

valid:   0%|          | 0/600 [00:00<?, ?it/s]


                === 1th Epoch ===
    
        Train Loss : 3.89 | Train Acc : 0.143
        Valid Loss : 3.412 | Valid Acc : 0.211
        
    


train:   0%|          | 0/600 [00:00<?, ?it/s]

valid:   0%|          | 0/600 [00:00<?, ?it/s]

...

valid:   0%|          | 0/600 [00:00<?, ?it/s]


                === 19th Epoch ===
    
        Train Loss : 0.3 | Train Acc : 0.922
        Valid Loss : 0.226 | Valid Acc : 0.94
        
    


train:   0%|          | 0/600 [00:00<?, ?it/s]

valid:   0%|          | 0/600 [00:00<?, ?it/s]


                === 20th Epoch ===
    
        Train Loss : 0.322 | Train Acc : 0.92
        Valid Loss : 0.26 | Valid Acc : 0.933
        
    
